<a href="https://colab.research.google.com/github/AlekseiChumachenko/diploma_dsu_34/blob/main/%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем датасет для обучения модели

In [ ]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2024-04-09 17:38:52--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.01s   

2024-04-09 17:38:53 (22.7 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [ ]:
!unzip WiC_dataset.zip

Archive:  WiC_dataset.zip
   creating: dev/
  inflating: dev/dev.data.txt        
  inflating: dev/dev.gold.txt        
   creating: test/
  inflating: test/test.gold.txt      
  inflating: test/test.data.txt      
   creating: train/
  inflating: train/train.data.txt    
  inflating: train/train.gold.txt    
  inflating: README.txt              


In [ ]:
with open('/content/train/train.data.txt', "r") as train_data:
  train_dataset = train_data.readlines()

In [ ]:
with open('/content/train/train.gold.txt', "r") as train_lb:
  train_labels = train_lb.readlines()

In [ ]:
with open('/content/test/test.data.txt', "r") as test_data:
  test_dataset = test_data.readlines()

In [ ]:
with open('/content/test/test.gold.txt', "r") as test_lb:
  test_labels = test_lb.readlines()

In [ ]:
train_dataset[0]

'carry\tV\t2-1\tYou must carry your camping gear .\tSound carries well over water .\n'

In [ ]:
train_labels[0][0]

'F'

Импортируем энкодер

In [ ]:
!git clone https://github.com/pierluigic/xl-lexeme.git

Cloning into 'xl-lexeme'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 244 (delta 34), reused 49 (delta 19), pack-reused 169
Receiving objects: 100% (244/244), 128.60 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (141/141), done.


Импортируем все необходимые библиотеки

In [ ]:
import os
import math
from scipy.spatial import distance
import numpy as np
import torch
import torch.nn as nn
import time
import pickle
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import json
import glob
import re

In [ ]:
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 980.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1

In [ ]:
os.chdir('/content/xl-lexeme')

In [ ]:
from WordTransformer import WordTransformer,InputExample

model = WordTransformer("xlm-roberta-large")

# examples = InputExample(texts="the quick fox jumps over the lazy dog", positions=[10,13])
# fox_embedding = model.encode(examples) #The embedding of the target word "fox"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/623 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

onnx/tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Импорт модели оценки близости значений

In [ ]:
!pip install metric-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
import metric_learn

Парсинг датасета для подачи в энкодер

In [ ]:
def parser (data_set, labels_set):
  data = []
  labels = []
  for i in range(len(data_set)):
    t = data_set[i].split('\t')
    s1=0
    for i1 in t[3].split(' ')[:int(t[2].split('-')[0])+1]:
      s1+=len(i1)+1
    example1 = InputExample(texts=t[3], positions=[s1-1-len(t[3].split(' ')[int(t[2].split('-')[0])]),s1-1])
    s2=0
    for i2 in t[4].split(' ')[:int(t[2].split('-')[1])+1]:
      s2+=len(i2)+1
    example2 = InputExample(texts=t[4], positions=[s2-1-len(t[4].split(' ')[int(t[2].split('-')[1])]),s2-1])
    if labels_set[i][0]=='F':
      y = -1
    else:
      y = 1
    ds=(example1, example2)
    data.append((ds,y))
  return data

In [ ]:
train_data = parser(train_dataset, train_labels)

In [ ]:
test_data = parser(test_dataset, test_labels)

Получение эмбедингов с помощью Siamese biencoder

In [ ]:
def get_prepared_data(row_data):
  """
  row_data is a list of tuples, where the first element is tuple of InputExamples,
  and the second element is label (int -1 or 1)
  """
  vecs = []
  labels = []
  for d,l in tqdm(row_data):
    vecs.append(model.encode([d[0],d[1]]))
    labels.append(l)
  return vecs,labels

In [ ]:
prepared_data = get_prepared_data(train_data)

100%|██████████| 5428/5428 [1:17:44<00:00,  1.16it/s]


In [ ]:
test_prepared_data = get_prepared_data(test_data)

100%|██████████| 1400/1400 [20:02<00:00,  1.16it/s]


Сохраняем полученные эмбединги

In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
# with open("prepared_data.pickle", "wb") as file:
#     pickle.dump(prepared_data, file)

In [ ]:
with open("prepared_data.pickle", "rb") as file:
    prepared_data = pickle.load(file)

In [ ]:
# with open("test_prepared_data.pickle", "wb") as file:
#     pickle.dump(test_prepared_data, file)

In [ ]:
with open("/content/drive/MyDrive/test_prepared_data.pickle", "rb") as file:
    test_prepared_data = pickle.load(file)

In [ ]:
os.chdir('/content/drive/MyDrive/itml_models')

Тренируем модель на полученных эмбедингах и сохраняем полученные модели. Тренируем модели с разными значениями параметра gamma - от 0.000001 до 10

In [ ]:
def train_itml(pairs, lbls):
  for degree in range(-5, 2):
    gamma = 10**degree
    itml = metric_learn.ITML(gamma=gamma)
    vecs, lbl = pairs, lbls
    itml.fit(vecs,lbl)
    pickle.dump(itml, open(f'itml_gamma={gamma}.plk', 'wb'))

In [ ]:
train_itml(prepared_data[0], prepared_data[1])

/usr/local/lib/python3.10/dist-packages/metric_learn/itml.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = min(_lambda[i], gamma_proj * (1. / wtw - 1. / pos_bhat[i]))
/usr/local/lib/python3.10/dist-packages/metric_learn/itml.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = min(_lambda[i], gamma_proj * (1. / wtw - 1. / pos_bhat[i]))
/usr/local/lib/python3.10/dist-packages/metric_learn/itml.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = min(_lambda[i], gamma_proj * (1. / wtw - 1. / pos_bhat[i]))
/usr/local/lib/python3.10/dist-packages/metric_learn/itml.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = min(_lambda[i], gamma_proj * (1. / wtw - 1. / pos_bhat[i]))
/usr/local/lib/python3.10/dist-packages/metric_learn/itml.py:75: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = min(_lambda[i], gamma_proj * (1. / wtw - 1. / pos_bhat[i]))
/usr/local/lib/python3.10

Функция оценки качества модели

In [ ]:
def evaluate_model(model_name, data, labels):
  with open(model_name, "rb") as file:
    itml = pickle.load(file)
    lbls_pred = itml.predict(data)
    score = accuracy_score(labels, lbls_pred)
    return score

In [ ]:
models_path = '/content/drive/MyDrive/itml_models'
for degree in range(-5, 2):
  gamma = 10**degree
  names = models_path+"/itml_gamma="+str(gamma)+".plk"
  print(f'score itml_gamma={gamma}=',evaluate_model(model_name = names, data=test_prepared_data[0], labels=test_prepared_data[1]))

score itml_gamma=1e-05= 0.5442857142857143
score itml_gamma=0.0001= 0.5464285714285714
score itml_gamma=0.001= 0.54
score itml_gamma=0.01= 0.5407142857142857
score itml_gamma=0.1= 0.5735714285714286
score itml_gamma=1= 0.5935714285714285
score itml_gamma=10= 0.575


Сравним с метками на основе косинусного расстояния

In [ ]:
cos_labels = []
for pairs in test_prepared_data[0]:
  dist = distance.cosine(pairs[0], pairs[1])
  if 0.5 <= dist <=1.5:
    lb = -1
  else:
    lb = 1
  cos_labels.append(lb)

In [ ]:
print(f'score cosine distance',accuracy_score(test_prepared_data[1], cos_labels))

score cosine distance 0.5


Для английского языка точность модели semantic change detection выше, чем при использовании косинусного расстояния

WiC для русского языка

Считываем тренировочные данные для русского языка

In [ ]:
with open ('/content/drive/MyDrive/WiC Russian/train.jsonl', 'r') as file:
  train_rus = file.read()

In [ ]:
wic_rus_train = []
for wic_dict in train_rus.split('\n'):
  try:
    wic_rus_train.append(json.loads(wic_dict))
  except:
    pass

Функция пасинга данных для русского языка (формат данных отличается)

In [ ]:
def parser_dict (list_of_dicts):
  data = []
  for d in list_of_dicts:
    example1 = InputExample(texts=d['sentence1'],positions=[d['start1'],d['end1']])
    example2 = InputExample(texts=d['sentence2'],positions=[d['start2'],d['end2']])
    if d['label']==1:
      y = 1
    else:
      y = -1
    ds=(example1, example2)
    data.append((ds,y))
  return data

In [ ]:
rus_train_data =  parser_dict(wic_rus_train)

In [ ]:
rus_prepared_data = get_prepared_data(rus_train_data)

100%|██████████| 19845/19845 [6:20:25<00:00,  1.15s/it]


Сохраняем эмбелинги

In [ ]:
# with open("/content/drive/MyDrive/rus_prepared_data.pickle", "wb") as file:
#     pickle.dump(rus_prepared_data, file)

In [ ]:
with open("/content/drive/MyDrive/rus_prepared_data.pickle", "rb") as file:
    rus_prepared_data = pickle.load(file)

In [ ]:
os.chdir('/content/drive/MyDrive/itml_models_rus')

Тренируем модель для русского языка

In [ ]:
train_itml(rus_prepared_data[0], rus_prepared_data[1])

In [ ]:
for degree in range(-5, 0):
  gamma = 10**degree
  itml = metric_learn.ITML(gamma=gamma)
  vecs, lbl = rus_prepared_data[0], rus_prepared_data[1]
  itml.fit(vecs,lbl)
  pickle.dump(itml, open(f'/content/drive/MyDrive/itml_models_rus/itml_gamma={gamma}.plk', 'wb'))

Подготовка тестовых данных

In [ ]:
with open ('/content/drive/MyDrive/WiC Russian/val.jsonl', 'r') as file:
  val_rus = file.read()

In [ ]:
wic_rus_val = []
for wic_dict in val_rus.split('\n'):
  try:
    wic_rus_val.append(json.loads(wic_dict))
  except:
    pass

In [ ]:
rus_val_data =  parser_dict(wic_rus_val)

In [ ]:
rus_prepared_data_val = get_prepared_data(rus_val_data)

100%|██████████| 8505/8505 [3:12:01<00:00,  1.35s/it]


In [ ]:
os.chdir('/content/drive/MyDrive')

In [ ]:
# with open("rus_prepared_data_val.pickle", "wb") as file:
#     pickle.dump(rus_prepared_data_val, file)

In [ ]:
with open("/content/drive/MyDrive/rus_prepared_data_val.pickle", "rb") as file:
    rus_prepared_data_val = pickle.load(file)

Оценка качества модели

In [ ]:
models_path = '/content/drive/MyDrive/itml_models_rus'
for degree in range(-5, 0):
  gamma = 10**degree
  names = models_path+"/itml_gamma="+str(gamma)+".plk"
  print(f'score itml_gamma={gamma}=',evaluate_model(model_name = names, data=rus_prepared_data_val[0], labels=rus_prepared_data_val[1]))

score itml_gamma=1e-05= 0.6320987654320988
score itml_gamma=0.0001= 0.6338624338624339
score itml_gamma=0.001= 0.6353909465020576
score itml_gamma=0.01= 0.6378600823045267
score itml_gamma=0.1= 0.660082304526749


Сравним с метками на основе косинусного расстояния

In [ ]:
cos_labels_rus = []
for pairs in rus_prepared_data_val[0]:
  dist = distance.cosine(pairs[0], pairs[1])
  if 0.5 <= dist <=1.5:
    lb = -1
  else:
    lb = 1
  cos_labels_rus.append(lb)

In [ ]:
print(f'score cosine distance for russian words',accuracy_score(rus_prepared_data_val[1], cos_labels_rus))

score cosine distance for russian words 0.36907701352145794


Для русского языка точность гораздо выше. Возможно из-за того, что тренировочный датассет гораздо больше и модель обучилась лучше

Посмотрим, как работает на задаче Rushifteval

Загружаем модель с наивысшей точностью предсказаний

In [ ]:
with open('/content/drive/MyDrive/itml_models_rus/itml_gamma=0.1.plk', "rb") as file:
  itml = pickle.load(file)

Подготовим данные. В данной задаче необходимо сравнить семантические отличия слов для случаев употребления в разные исторические периоды. Например, как изменялось значение слов в перод 1918-1990гг по сравнению с периодом 1700-1916гг

In [ ]:
data_dir = f"/content/drive/MyDrive/rushifteval/rushifteval1"
data_words_paths = glob.glob(f"{data_dir}/*/uses.csv")
rus_words = [path.split('/')[-2] for path in data_words_paths]

In [ ]:
words_to_encode ={}
data_dir = f"/content/drive/MyDrive/rushifteval/rushifteval1"
data_words_paths = glob.glob(f"{data_dir}/*/uses.csv")
rus_words = [path.split('/')[-2] for path in data_words_paths]
for i in range(len(data_words_paths)):
    words_to_encode[data_words_paths[i].split('/')[-2]] = {}
    with open(data_words_paths[i], "r") as file:
      line = file.readlines()
      group = []
      for ln in line[1:]:
        group.append(ln.split('\t')[3])
      group = (group)
      for s in group:
        words_to_encode[data_words_paths[i].split('/')[-2]][s] = []
      for ln in line[1:]:
        words_to_encode[data_words_paths[i].split('/')[-2]][ln.split('\t')[3]].append(InputExample(texts=ln.split('\t')[6], positions=[int(ln.split('\t')[7].split(':')[0]),int(ln.split('\t')[7].split(':')[1])]))


In [ ]:
words_metrics_1_ = {}
words_metrics_2_ = {}
i = 0
for k in tqdm(words_to_encode.keys()):
  words_metrics_1_[k] = 0
  words_metrics_2_[k] = 0
  groups = []
  for g in words_to_encode[k].keys():
    groups.append(g)
  words_metrics_1_[k] = model.encode(words_to_encode[k][groups[0]])
  words_metrics_2_[k] = model.encode(words_to_encode[k][groups[1]])
with open("/content/drive/MyDrive/words_metrics_1_.pickle", "wb") as file:
    pickle.dump(words_metrics_1_, file)
with open("/content/drive/MyDrive/words_metrics_2_.pickle", "wb") as file:
    pickle.dump(words_metrics_2_, file)

100%|██████████| 111/111 [4:15:24<00:00, 138.06s/it]


In [ ]:
with open("/content/drive/MyDrive/words_metrics_1_.pickle", "rb") as file:
    words_metrics_1_ = pickle.load(file)
with open("/content/drive/MyDrive/words_metrics_2_.pickle", "rb") as file:
    words_metrics_2_ = pickle.load(file)

Для каждого слова сравниваем попарно все его употребления в разные периоды и выводим среднее значение метрики simantic change distance

In [ ]:
metric = itml.get_metric()
word_dist = {}
for target_word in tqdm(rus_words):
  count = 0
  dist_sum = 0
  for word_vec_1 in words_metrics_1_[target_word]:
    for word_vec_2 in words_metrics_2_[target_word]:
      dist = metric(word_vec_1, word_vec_2)
      dist_sum += dist
      count += 1
  word_dist[target_word] = dist_sum/count
print(word_dist)

100%|██████████| 111/111 [01:08<00:00,  1.62it/s]

{'апостол': 1.5020183725211094, 'веер': 1.3934205916446305, 'бригада': 1.4252952883747998, 'блин': 1.3750609378719414, 'благодарность': 1.432597731351687, 'блондин': 1.3153430697931792, 'брат': 1.419599283336067, 'век': 1.4346139821229669, 'амбиция': 1.3994306234577132, 'авторитет': 1.408091398230734, 'дух': 1.4475107968145033, 'возраст': 1.4390783327155008, 'железо': 1.3966482590043798, 'жесть': 1.4032616107368627, 'головка': 1.397857454089018, 'дядька': 1.3577066099293027, 'вызов': 1.4384923091439068, 'грех': 1.4219263460556337, 'дядя': 1.3713269360715066, 'верховье': 1.3651557264928529, 'издательство': 1.4653330962722164, 'кабан': 1.351390883730697, 'итальянец': 1.4292641683929128, 'завод': 1.4212732003762072, 'земля': 1.5491341902148923, 'крушение': 1.4226605747170487, 'карман': 1.3963206959632468, 'живот': 1.4398906574753088, 'заблуждение': 1.3782029337054487, 'закладка': 1.458158552357957, 'любовь': 1.4783408448833344, 'лишение': 1.442029531070698, 'маньяк': 1.3468917728422203, '

In [ ]:
print(max(list(word_dist.values())))
print(min(list(word_dist.values())))

1.6432446640999006
1.2954883994658588


Посмотрим на эталонные оценки, выставленные экспертами. Оценки выставлялись следующим образом

*  0 - Трудно пределить
*  1 - Значения разные
*  2 - Значения сильно дистанцированы
*  3 - Значения близки
*  4 - Значения полностью идентичны








In [ ]:
with open('/content/drive/MyDrive/annotated_all.tsv', 'r') as file:
  gold_metrics = file.readlines()

In [ ]:
gold = {}
for l in gold_metrics:
  gold[l.split('\t')[0]] = float(l.split('\t')[1])
print(gold)

{'авторитет': 3.2333333333333334, 'амбиция': 3.111111111111111, 'апостол': 3.4941176470588236, 'благодарность': 3.2333333333333334, 'блин': 3.2134831460674156, 'блондин': 3.9444444444444446, 'брат': 3.2247191011235956, 'бригада': 2.8, 'веер': 2.550561797752809, 'век': 3.2, 'верховье': 3.689655172413793, 'возраст': 3.477777777777778, 'вызов': 2.1777777777777776, 'головка': 2.2045454545454546, 'грех': 3.488888888888889, 'дух': 2.3222222222222224, 'дядька': 2.5930232558139537, 'дядя': 3.3777777777777778, 'железо': 2.2, 'жесть': 3.235955056179775, 'живот': 2.911111111111111, 'заблуждение': 3.5, 'завод': 3.226086956521739, 'закладка': 1.9333333333333333, 'земля': 2.8392857142857144, 'издательство': 3.533333333333333, 'итальянец': 3.707865168539326, 'кабан': 3.6, 'карман': 3.466666666666667, 'крушение': 2.752808988764045, 'крыша': 3.574712643678161, 'кулиса': 3.1666666666666665, 'лечение': 3.6555555555555554, 'линейка': 1.879120879120879, 'лишение': 2.9444444444444446, 'локоть': 3.2777777777

Посмотрим на корреляцию

In [ ]:
trues = []
preds = []
for target_word in tqdm(rus_words):
  try:
    trues.append(gold[target_word])
    preds.append(word_dist[target_word])
  except:
    pass

100%|██████████| 111/111 [00:00<00:00, 64367.17it/s]


In [ ]:
res = spearmanr(np.array(trues), np.array(preds))

In [ ]:
res

SignificanceResult(statistic=-0.4356792104582177, pvalue=2.728270337733736e-06)

Есть отрицательная корреляция - чем выше simantic change distance - тем ниже долны быть оценки

Посмотрим детально

Для каждого слова посчитаем среднее значение метки "одинаковости" при его использовании в разные эпохи

In [ ]:
metric = itml.get_metric()
word_dist = {}
for target_word in tqdm(rus_words):
  count = 0
  lbl_count = []
  for word_vec_1 in words_metrics_1_[target_word]:
    for word_vec_2 in words_metrics_2_[target_word]:
        lbl = itml.predict(np.array([word_vec_1,word_vec_2]).reshape(1,2,-1))
        lbl_count.append(lbl)
  word_dist[target_word] = lbl_count

100%|██████████| 111/111 [01:37<00:00,  1.14it/s]


In [ ]:
for k in word_dist.keys():
  print(k, '=',np.array(word_dist[k]).mean())

апостол = -0.7455410225921522
веер = -0.4666666666666667
бригада = -0.4288888888888889
блин = -0.3333333333333333
благодарность = -0.5866666666666667
блондин = -0.1
брат = -0.5666666666666667
век = -0.5957193816884662
амбиция = -0.4888888888888889
авторитет = -0.5466666666666666
дух = -0.7577777777777778
возраст = -0.7222222222222222
железо = -0.4822222222222222
жесть = -0.29555555555555557
головка = -0.5111111111111111
дядька = -0.3460166468489893
вызов = -0.6377777777777778
грех = -0.4822222222222222
дядя = -0.36444444444444446
верховье = -0.17333333333333334
издательство = -0.4888888888888889
кабан = -0.21555555555555556
итальянец = -0.5933333333333334
завод = -0.5711111111111111
земля = -0.7444444444444445
крушение = -0.6
карман = -0.4266666666666667
живот = -0.4955555555555556
заблуждение = -0.33111111111111113
закладка = -0.7155555555555555
любовь = -0.6333333333333333
лишение = -0.5955555555555555
маньяк = -0.24222222222222223
лечение = -0.6888888888888889
лох = -0.7037037037037

Для конкретного слова, например, "цензура". Средняя экспертная оценка: 3.49438202247191, то есть значение практически не поменялось. Средняя метка модели: -0.2288888888888889 - близка к нулю, что также говорит о близости значений.


Тем временем для слова "частица" оценка экспертов 1.9666666666666666 - значение сильно поменялось. Средняя метка на основе семантического расстояния равна -0.82, что также говорит о сильном изменении значения слова.

Теперь посмотрим работу на задаче Semeval

Зашружаем данные

In [ ]:
with open('/content/drive/MyDrive/semeval/ccoha1.txt', 'r') as file:
  ccoha1 = file.readlines()

In [ ]:
with open('/content/drive/MyDrive/semeval/ccoha2.txt', 'r') as file:
  ccoha2 = file.readlines()

In [ ]:
with open('/content/drive/MyDrive/semeval/lem_ccoha1.txt', 'r') as file:
  lem_ccoha1 = file.readlines()

In [ ]:
with open('/content/drive/MyDrive/semeval/lem_ccoha2.txt', 'r') as file:
  lem_ccoha2 = file.readlines()

In [ ]:
with open('/content/drive/MyDrive/semeval/targets.txt', 'r') as file:
  targets = file.readlines()

In [ ]:
with open('/content/drive/MyDrive/semeval/graded.txt', 'r') as file:
  graded = file.readlines()

In [ ]:
targets = [wrd.split('_')[0] for wrd in targets]

In [ ]:
graded_dict = {}
for line in graded:
  graded_dict[line.split('\t')[0].split('_')[0]]=float(line.split('\t')[1])
print(graded_dict)

{'attack': 0.1439699927, 'bag': 0.1003636619, 'ball': 0.4093665525, 'bit': 0.3065766263, 'chairman': 0.0, 'circle': 0.1710871199, 'contemplation': 0.070838721, 'donkey': 0.1601038896, 'edge': 0.2609655868, 'face': 0.137791275, 'fiction': 0.0207232682, 'gas': 0.1595695865, 'graft': 0.5539755971, 'head': 0.2952561247, 'land': 0.2234476353, 'lane': 0.1037200629, 'lass': 0.2125896937, 'multitude': 0.1003636619, 'ounce': 0.2848990681, 'part': 0.161270809, 'pin': 0.2072118595, 'plane': 0.8823475834, 'player': 0.2736674651, 'prop': 0.624759971, 'quilt': 0.1231453661, 'rag': 0.2765152931, 'record': 0.4273504707, 'relationship': 0.0562177896, 'risk': 0.0, 'savage': 0.0968689424, 'stab': 0.4005896829, 'stroke': 0.1762311532, 'thump': 0.1429915711, 'tip': 0.6788987615, 'tree': 0.070838721, 'twist': 0.3984926634, 'word': 0.1793071623}


Функция парсинга данных

In [ ]:
def parser_semeval(lem_ccoha_list, ccoha_list, targets):
  text = {}
  positions = {}
  ind_sentence = {}
  ind_word = {}
  for w in targets:
    text[w] = []
    positions[w] = []
    ind_sentence[w] = []
    ind_word[w] = []
    for sentence in lem_ccoha_list:
      p1 = 0
      p2 = 0
      if w in sentence.split(' '):
        text[w].append(sentence)
        for word in sentence.split(' ')[:sentence.split(' ').index(w)+1]:
          p2+=len(word) + 1
          p1 = p2-len(word)
        positions[w].append((p1-1,p2-1))
  return text, positions

In [ ]:
sentence_1, positions_1 = parser_semeval(lem_ccoha1,ccoha1,targets)

In [ ]:
sentence_2, positions_2 = parser_semeval(lem_ccoha2,ccoha2,targets)

Получаем эмбединги

In [ ]:
def encoder_semeval (s_1, s_2, p_1, p_2):
  data_1 = {}
  data_2 = {}
  for d in tqdm(s_1.keys()):
    data_1[d] = 0
    data_2[d] = 0
    examples1 = []
    examples2 = []
    for num, sample in enumerate(s_1[d]):
      example1 = InputExample(texts=sample, positions=[p_1[d][num][0],p_1[d][num][1]])
      examples1.append(example1)
    data_1[d] = model.encode(examples1)
    for num, sample in enumerate(s_2[d]):
      example2 = InputExample(texts=sample, positions=[p_2[d][num][0],p_2[d][num][1]])
      examples2.append(example2)
    data_2[d] = model.encode(examples2)
  return data_1, data_2

In [ ]:
encoded_corp_1, encoded_corp_2 = encoder_semeval(sentence_1,sentence_2,positions_1,positions_2)

100%|██████████| 37/37 [2:23:28<00:00, 232.67s/it]


Сохраняем полученные эмбединги

In [ ]:
with open("/content/drive/MyDrive/semeval/encoded_corp_1.pickle", "wb") as file:
    pickle.dump(encoded_corp_1, file)
with open("/content/drive/MyDrive/semeval/encoded_corp_2.pickle", "wb") as file:
    pickle.dump(encoded_corp_2, file)

In [ ]:
with open('/content/drive/MyDrive/semeval/encoded_corp_1.pickle', 'rb') as file:
  encoded_corp_1 = pickle.load(file)
with open('/content/drive/MyDrive/semeval/encoded_corp_2.pickle', 'rb') as file:
  encoded_corp_2 = pickle.load(file)

Загружаем модель с наилучшими параметрами

In [ ]:
with open('/content/drive/MyDrive/itml_models/itml_gamma=0.1.plk', "rb") as file:
  itml = pickle.load(file)

Посмотрим, как отличаются значения слов в разных корпусах текстов

In [ ]:
res = {}
for d in tqdm(encoded_corp_1.keys()):
  res[d] = []
  for i in range(len(encoded_corp_1[d])):
    for j in range(len(encoded_corp_2[d])):
      res[d].append(itml.predict(np.array([encoded_corp_1[d][i],encoded_corp_2[d][j]]).reshape(1,2,-1)))

100%|██████████| 37/37 [22:40<00:00, 36.77s/it]


Средние значения получившихся оценок для каждого слова

In [ ]:
for word in res.keys():
  for i, lbl in enumerate(res[word]):
    if lbl == -1:
      res[word][i] = 1
    else:
      res[word][i] = 0
  print(word, '=', np.array([res[word]]).mean())

attack = 0.9708210784313726
bag = 0.9714285714285714
ball = 0.9947916666666666
bit = 0.9296952681757034
chairman = 0.84
circle = 0.9645881782945737
contemplation = nan
donkey = nan
edge = 0.9823232323232324


<ipython-input-7-b90b6464241b>:7: RuntimeWarning: Mean of empty slice.
  print(word, '=', np.array([res[word]]).mean())
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


face = 0.9813527066760467
fiction = nan
gas = nan
graft = 0.984313725490196
head = 0.9860421735604218
land = 0.9702172096908939
lane = 0.8333333333333334
lass = nan
multitude = nan
ounce = nan
part = 0.9718833711106531
pin = 0.9798711755233495
plane = 0.9583333333333334
player = nan
prop = 0.977656405163853
quilt = 0.98
rag = 1.0
record = 0.9841896590946021
relationship = nan
risk = 0.9380635624538064
savage = 0.950894538606403
stab = 0.9534465020576132
stroke = 0.9754975310489301
thump = 0.9358288770053476
tip = 0.978787498431028
tree = 1.0
twist = 0.9731967992837558
word = 0.9423076923076923


Для всех слов модель посчитала, что значения в для двух корпусов предложений отличаются

Сравним с эталоном

In [ ]:
with open('/content/drive/MyDrive/semeval/binary.txt', 'r') as file:
  binary = file.readlines()
for line in binary:
  print(line.split('\t')[0].split('_')[0], " - ", line.split('\t')[1][0])

attack  -  1
bag  -  0
ball  -  0
bit  -  1
chairman  -  0
circle  -  1
contemplation  -  0
donkey  -  0
edge  -  1
face  -  0
fiction  -  0
gas  -  0
graft  -  1
head  -  1
land  -  1
lane  -  0
lass  -  1
multitude  -  0
ounce  -  0
part  -  0
pin  -  0
plane  -  1
player  -  1
prop  -  1
quilt  -  0
rag  -  1
record  -  1
relationship  -  0
risk  -  0
savage  -  0
stab  -  1
stroke  -  0
thump  -  1
tip  -  1
tree  -  0
twist  -  0
word  -  0


С помощью коэффициента корреляции сравним метрики близости для полученной модели и эталона.

In [ ]:
metric = itml.get_metric()
word_dist_semeval = {}
for target_word in tqdm(targets):
  count = 0
  dist_sum = 0
  for word_vec_1 in encoded_corp_1[target_word]:
    for word_vec_2 in encoded_corp_2[target_word]:
      dist = metric(word_vec_1, word_vec_2)
      dist_sum += dist
      count += 1
  try:
    word_dist_semeval[target_word] = dist_sum/count
  except:
    pass
print(word_dist_semeval)

100%|██████████| 37/37 [09:55<00:00, 16.09s/it]

{'attack': 1.5136992352158156, 'bag': 1.5270193883465162, 'ball': 1.5596091426653282, 'bit': 1.4407684607900397, 'chairman': 1.4080480860135176, 'circle': 1.5308094813428466, 'edge': 1.6024887463131854, 'face': 1.5635563131722674, 'graft': 1.5466858755333281, 'head': 1.5635658933169942, 'land': 1.5206968061034383, 'lane': 1.3543607904973893, 'part': 1.532328568299047, 'pin': 1.5764486042632981, 'plane': 1.4398314637015617, 'prop': 1.5717406837397612, 'quilt': 1.5388396104463464, 'rag': 1.4931494328465151, 'record': 1.557592911592575, 'risk': 1.4594642847246206, 'savage': 1.4768165223490333, 'stab': 1.4881632564530916, 'stroke': 1.5453796263988977, 'thump': 1.4824651297779126, 'tip': 1.545527944783831, 'tree': 1.731240437337663, 'twist': 1.5480308111969565, 'word': 1.481131896836426}


In [ ]:
trues = []
preds = []
for target_word in tqdm(word_dist_semeval.keys()):
    trues.append(graded_dict[target_word])
    preds.append(word_dist_semeval[target_word])

100%|██████████| 28/28 [00:00<00:00, 141665.27it/s]


In [ ]:
res = spearmanr(np.array(trues), np.array(preds))
res

SignificanceResult(statistic=0.2805528971120218, pvalue=0.14815160592833443)